In [28]:
import pandas as pd
from scipy import stats
import numpy as np

# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

Это задание поможет вам лучше разобраться в методах множественной проверки гипотез и позволит применить ваши знания на данных из реального биологического исследования.
В этом задании вы:

   * вспомните, что такое t-критерий Стьюдента и для чего он применяется
   * сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
   * почувствуете разницу в результатах применения различных методов поправки на множественную проверку

## Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

In [5]:
gene_data = pd.read_csv('gene_high_throughput_sequencing.csv', sep = ',', header = 0)

In [24]:
gene_data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [9]:
gene_data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

## Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

Fc(C,T)=T/C,T>C   
Fc(C,T)=−C/T,T<C

где C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

   * для групп normal (control) и early neoplasia (treatment)
   * для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [25]:
gene_data_normal = gene_data[gene_data.Diagnosis == 'normal'].drop(['Patient_id', 'Diagnosis'], axis = 1)
gene_data_early_neoplasia = gene_data[gene_data.Diagnosis == 'early neoplasia'].drop(['Patient_id', 'Diagnosis'], axis = 1)
gene_data_cancer = gene_data[gene_data.Diagnosis == 'cancer'].drop(['Patient_id', 'Diagnosis'], axis = 1)

In [96]:
p_values1 = []
p_values2 = []
for i in range(gene_data_normal.shape[1]):
    p_values1.append(stats.ttest_ind(gene_data_normal.iloc[:,i], gene_data_early_neoplasia.iloc[:,i], equal_var = False)[1])
    p_values2.append(stats.ttest_ind(gene_data_early_neoplasia.iloc[:,i], gene_data_cancer.iloc[:,i], equal_var = False)[1])

In [97]:
table = pd.DataFrame()
table['p_val(normal - neop)'] = p_values1
table['p_val(neop - cancer)'] = p_values2

In [98]:
answer1 = len(table[table['p_val(normal - neop)'] < 0.05])

In [99]:
answer2 = len(table[table['p_val(neop - cancer)'] < 0.05])

In [69]:
def write_answer_1(answer):
    with open("gene_answer_1.txt", "w") as fout:
        fout.write(str(answer))
        
def write_answer_2(answer):
    with open("gene_answer_2.txt", "w") as fout:
        fout.write(str(answer))

In [105]:
write_answer_1(answer1)
write_answer_2(answer2)

### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

Обратите внимание, что

   * применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
   * при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)! 

In [83]:
def calculate_fold_change(control, treatment):
    C = np.mean(control)
    T = np.mean(treatment)
    if T > C:
        fold_change = T/C
    else:
        fold_change = -C/T
    return fold_change

In [84]:
fold_change1 = []
fold_change2 = []
for i in range(gene_data_normal.shape[1]):
    fold_change1.append(calculate_fold_change(gene_data_normal.iloc[:,i], gene_data_early_neoplasia.iloc[:,i]))
    fold_change2.append(calculate_fold_change(gene_data_early_neoplasia.iloc[:,i], gene_data_cancer.iloc[:,i]))

In [101]:
table['fold_change(normal - neop)'] = fold_change1
table['fold_change(neop - cancer)'] = fold_change2

In [103]:
table.head()

,p_val(normal - neop),p_val(neop - cancer),fold_change(normal - neop),fold_change(neop - cancer)
0,0.690766,0.413735,-1.067858,-1.148593
1,0.000032,0.653429,1.996517,-1.064871
2,0.060273,0.079556,1.222418,1.278517
3,0.826429,0.287581,-1.020070,1.092635
4,0.049876,0.463292,1.125471,1.036574


In [107]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [151]:
reject, p_corrected1, a1, a2 = multipletests(p_values1, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [152]:
reject, p_corrected2, a1, a2 = multipletests(p_values2, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [153]:
table['p_val(normal - neop)'] = p_corrected1
table['p_val(neop - cancer)'] = p_corrected2

In [154]:
answer3 = len(table[(table['p_val(normal - neop)'] < 0.025) & (abs(table['fold_change(normal - neop)']) > 1.5)])
answer4 = len(table[(table['p_val(neop - cancer)'] < 0.025) & (abs(table['fold_change(neop - cancer)']) > 1.5)])

In [117]:
def write_answer_3(answer):
    with open("gene_answer_3.txt", "w") as fout:
        fout.write(str(answer))
        
def write_answer_4(answer):
    with open("gene_answer_4.txt", "w") as fout:
        fout.write(str(answer))

In [159]:
write_answer_3(answer3)
write_answer_4(answer4)

### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

P.S. Вспомните, какое значение имеет уровень значимости α в каждой из поправок: Холма и Бенджамини-Хохберга. Одинаковый ли смысл имеет уровень значимости в каждой из поправок?

In [160]:
reject, p_corrected1, a1, a2 = multipletests(p_values1, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

reject, p_corrected2, a1, a2 = multipletests(p_values2, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [161]:
table['p_val(normal - neop)'] = p_corrected1
table['p_val(neop - cancer)'] = p_corrected2

In [172]:
answer5 = len(table[(table['p_val(normal - neop)'] < 0.025) & (abs(table['fold_change(normal - neop)']) > 1.5)])
answer6 = len(table[(table['p_val(neop - cancer)'] < 0.025) & (abs(table['fold_change(neop - cancer)']) > 1.5)])

In [173]:
def write_answer_5(answer):
    with open("gene_answer_5.txt", "w") as fout:
        fout.write(str(answer))
        
def write_answer_6(answer):
    with open("gene_answer_6.txt", "w") as fout:
        fout.write(str(answer))

In [174]:
write_answer_5(answer5)
write_answer_6(answer6)